In [ ]:
!pip install transformers datasets accelerate sentencepiece

In [ ]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast

model_name = "Ahmad/parsT5-base"

tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("AliFartout/PEYMA-ARMAN-Mixed")
dataset


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.31M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/431k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/423k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26384 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3296 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3296 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_names'],
        num_rows: 26384
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_names'],
        num_rows: 3296
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_names'],
        num_rows: 3296
    })
})

In [ ]:
def convert_to_t5(batch):
    tokens = batch["tokens"]
    labels = batch["ner_tags_names"]  # already strings
    sentence = " ".join(tokens)
    target = " ".join(labels)

    batch["input_text"] = f"ner: {sentence}"
    batch["target_text"] = target
    return batch

dataset = dataset.map(convert_to_t5)


Map:   0%|          | 0/26384 [00:00<?, ? examples/s]

Map:   0%|          | 0/3296 [00:00<?, ? examples/s]

Map:   0%|          | 0/3296 [00:00<?, ? examples/s]

In [ ]:
def tokenize(batch):
    model_inputs = tokenizer(
        batch["input_text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

    labels = tokenizer(
        batch["target_text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )["input_ids"]

    # Replace pad token IDs with -100 for loss
    labels = [
        (l if l != tokenizer.pad_token_id else -100)
        for l in labels
    ]
    model_inputs["labels"] = labels
    return model_inputs

tokenized = dataset.map(tokenize, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/26384 [00:00<?, ? examples/s]

Map:   0%|          | 0/3296 [00:00<?, ? examples/s]

Map:   0%|          | 0/3296 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/drive/MyDrive/parsT5-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1,          # keep only best model
    metric_for_best_model="loss",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,

    logging_steps=50,
    remove_unused_columns=False,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from transformers import Trainer, EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)


/tmp/ipython-input-453740440.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.250200,0.145525
2,0.169700,0.103554
3,0.139500,0.100613
4,0.128500,0.098302


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=13192, training_loss=0.8720626951420647, metrics={'train_runtime': 8227.1636, 'train_samples_per_second': 12.828, 'train_steps_per_second': 1.603, 'total_flos': 1.806508179534643e+16, 'train_loss': 0.8720626951420647, 'epoch': 4.0})

In [ ]:
test_results = trainer.evaluate(eval_dataset=tokenized["test"])
test_results


{'eval_loss': 0.10203143209218979,
 'eval_runtime': 82.2469,
 'eval_samples_per_second': 40.074,
 'eval_steps_per_second': 5.009,
 'epoch': 4.0}

In [ ]:
!pip install seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=c8ed2f15bba44f1731f2a7c3a0866c3112bdf368175145a423b620654129d4e7
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [ ]:

print(tokenized["train"].features)


print(dataset["train"].features)


{'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'labels': List(Value('int64'))}
{'tokens': List(Value('string')), 'ner_tags': List(ClassLabel(names=['B_LOC', 'I_DAT', 'B_PCT', 'I_LOC', 'I_PER', 'I_MON', 'B_ORG', 'B_PRO', 'B_PER', 'O', 'I_PCT', 'I_ORG', 'B_FAC', 'B_DAT', 'B_TIM', 'I_TIM', 'I_EVE', 'B_MON', 'I_PRO', 'B_EVE', 'I_FAC'])), 'ner_tags_names': List(Value('string')), 'input_text': Value('string'), 'target_text': Value('string')}


In [ ]:
ner_tag_names = dataset["train"].features["ner_tags"].feature.names
print(ner_tag_names)


['B_LOC', 'I_DAT', 'B_PCT', 'I_LOC', 'I_PER', 'I_MON', 'B_ORG', 'B_PRO', 'B_PER', 'O', 'I_PCT', 'I_ORG', 'B_FAC', 'B_DAT', 'B_TIM', 'I_TIM', 'I_EVE', 'B_MON', 'I_PRO', 'B_EVE', 'I_FAC']


In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from tqdm import tqdm
import torch

# Memory-efficient prediction function
def predict_in_batches(model, dataset, tokenizer, batch_size=16):
    model.eval()
    device = model.device

    all_predictions = []
    all_labels = []

    for i in tqdm(range(0, len(dataset), batch_size)):
        batch = dataset[i:i+batch_size]

        # Prepare inputs
        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)

        # Generate predictions
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128,
                num_beams=1,  # Use 1 for faster inference (greedy decoding)
            )

        # Decode predictions
        pred_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_predictions.extend(pred_texts)

        # Decode labels
        labels = batch["labels"]
        labels = [[l if l != -100 else tokenizer.pad_token_id for l in label] for label in labels]
        label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)
        all_labels.extend(label_texts)

        # Clear cache periodically
        if i % (batch_size * 10) == 0:
            torch.cuda.empty_cache()

    return all_predictions, all_labels

# Use the function
print("Generating predictions on test set...")
pred_texts, label_texts = predict_in_batches(
    model,
    tokenized["test"],
    tokenizer,
    batch_size=16  # Adjust based on your memory
)

# Parse into flat lists for classification report
pred_tags_list = []
true_tags_list = []

for pred, true in zip(pred_texts, label_texts):
    pred_tags = pred.split()
    true_tags = true.split()

    min_len = min(len(pred_tags), len(true_tags))
    pred_tags_list.extend(pred_tags[:min_len])
    true_tags_list.extend(true_tags[:min_len])

# Generate classification report
print("\n" + "="*80)
print("CLASSIFICATION REPORT")
print("="*80 + "\n")
print(classification_report(true_tags_list, pred_tags_list, digits=4))

# Overall accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_tags_list, pred_tags_list)
print(f"\nOverall Accuracy: {accuracy:.4f}")

Generating predictions on test set...


100%|██████████| 206/206 [09:14<00:00,  2.69s/it]



CLASSIFICATION REPORT



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

           b     0.0000    0.0000    0.0000         2
          b_     0.0000    0.0000    0.0000         4
       b_dat     0.0000    0.0000    0.0000       176
       b_eve     0.0000    0.0000    0.0000       216
       b_fac     0.0000    0.0000    0.0000       124
       b_loc     0.3969    0.0286    0.0533      1820
       b_mon     0.0000    0.0000    0.0000        50
       b_org     0.3051    0.0090    0.0175      1996
       b_pct     0.0000    0.0000    0.0000        26
       b_per     0.2269    0.0330    0.0575      1487
       b_pro     0.0000    0.0000    0.0000       261
       b_tim     0.0000    0.0000    0.0000        27
           i     0.0000    0.0000    0.0000         3
          i_     0.0000    0.0000    0.0000         4
       i_dat     0.0000    0.0000    0.0000       230
       i_eve     0.0000    0.0000    0.0000       691
       i_fac     0.0000    0.0000    0.0000       237
        i_lo     0.0000    